In [11]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
import warnings
import cv2 as cv

from typing import Union, Any, Tuple
from torch.utils.data import Dataset
from pathlib import Path
import os
import sys

warnings.filterwarnings("ignore")
torch.manual_seed(69)

home = os.getcwd()
current = os.getcwd()

while 'pytorch_modular' not in os.listdir(current):
    current = Path(current).parent

sys.path.append(str(current))
sys.path.append(os.path.join(str(current), 'pytorch_modular'))

In [14]:
TRAIN_DATA = os.path.join(home, 'data', 'train.csv')
TEST_DATA = os.path.join(home, 'data', 'test.csv')

## Data reading and preprocessing

Prepare a class for FashionMNIST dataset to easily read data and apply transformations when retrieving examples.

In [16]:

class TrainDs(Dataset):
    def __init__(self, train_data: Union[Path, str]) -> None:
        super().__init__()
        self.data = pd.read_csv(train_data, index_col=0)

    def __getitem__(self, index) -> Any:
        # get the image
        image_data, label = self.data.iloc[index, :-1].values, self.data.iloc[index, -1].item()
        assert isinstance(label, (int, float))
        # reshape the data and normalize
        image_data = image_data.reshape(28, 28, 1).astype(np.uint8)
        # convert to rgb
        image_data = cv.cvtColor(image_data, cv.COLOR_GRAY2BGR)
        # normalize
        image_data = image_data / 255.0

        return torch.from_numpy(image_data).permute(2, 0, 1).float(), label
    
    def __len__(self):
        return len(self.data) 


class TestDs(Dataset):
    def __init__(self, test_data: Union[Path, str]) -> None:
        super().__init__()
        self.data = pd.read_csv(test_data, index_col=0)

    def __getitem__(self, index) -> Any:
        # get the image
        image_data = self.data.iloc[index].values
    
        image_data = image_data.reshape(28, 28, 1).astype(np.uint8)
        # convert to rgb
        image_data = cv.cvtColor(image_data, cv.COLOR_GRAY2BGR)
        # normalize
        image_data = image_data / 255.0

        # make sure to permute the image dimensions to be compatible with pytorch
        return torch.from_numpy(image_data).permute(2, 0, 1).float()
           
    def __len__(self):
        return len(self.data) 
    
from sklearn.model_selection import train_test_split

train_ds = TrainDs(TRAIN_DATA)
train_ds, val_ds = train_test_split(train_ds, random_state=69, test_size=0.1)
test_ds = TestDs(test_data=TEST_DATA)


In [17]:
# let's create the data loaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_ds,
    batch_size=512,
    shuffle=True,
    num_workers=os.cpu_count() // 2,
    pin_memory=True,
    drop_last=True,
)

val_loader = DataLoader(
    val_ds,
    batch_size=512,
    shuffle=False,
    num_workers=os.cpu_count() // 2,
    pin_memory=True,
    drop_last=False,
)

test_loader = DataLoader(
    test_ds,
    batch_size=512,
    shuffle=False,
    num_workers=os.cpu_count() // 2,
    pin_memory=True,
    drop_last=False,
)

# Model

In [19]:
# let's build the model

from pytorch_modular.transfer_learning import resnetFeatureExtractor as rfe
from pytorch_modular.image_classification import classification_head as ch
from pytorch_modular.dimensions_analysis import dimension_analyser as da
import importlib

# importlib.reload(pytorch_modular)

importlib.reload(rfe)
importlib.reload(ch)
importlib.reload(da)

from torch import nn


class ClassificationModel(nn.Module):
    def __init__(self, 
                 feature_extractor, 
                 classification_head: ch.GenericClassifier,
                 input_shape: Tuple[int, int, int], 
                 num_classes: int,
                 *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.fe = feature_extractor
        self.head = classification_head
        analyser = da.DimensionsAnalyser()   

        # extract the number of channels
        c_index = np.argmin(input_shape)
        batched_input = tuple([dim for i, dim in enumerate(input_shape) if i != c_index])
        batched_input = (1, ) + batched_input + (input_shape[c_index], )

        # find the number of input features
        in_features = analyser.analyse_dimensions_static(feature_extractor, input_shape=batched_input)
        
        in_features = np.prod(in_features)

        self.head.in_features = in_features
        self.head.num_classes = num_classes
        
        self.model = nn.Sequential(self.fe, nn.Flatten(), self.head)

    def forward(self, x: torch.Tensor):
        return self.model.forward(x)

feature_extractor = rfe.ResNetFeatureExtractor(num_blocks=1, freeze=False)
head = ch.ExponentialClassifier(num_classes=10, in_features=1024, num_layers=4)
model = ClassificationModel(feature_extractor=feature_extractor, classification_head=head, input_shape=(1, 28, 28), num_classes=10)

In [28]:
from pytorch_modular.image_classification import engine_classification
from pytorch_modular.image_classification import epoch_engine
import pytorch_modular.exp_tracking as exp
importlib.reload(engine_classification)
importlib.reload(epoch_engine)
importlib.reload(exp)

from torch.optim import Adam
from torch.optim.lr_scheduler import LinearLR


optimizer = Adam(params=model.parameters(), lr=0.05)
scheduler = LinearLR(optimizer=optimizer, start_factor=1, end_factor=0.1, total_iters=100) 

train_configuration = {'optimizer': optimizer,
                        'scheduler': scheduler,
                        'min_val_loss': 10 ** -4,
                        'max_epochs': 25,
                        'report_epoch': 5,
                        }

r = engine_classification.train_model(model=model, 
                                  train_dataloader=train_dataloader, 
                                  test_dataloader=val_loader, 
                                  train_configuration=train_configuration, 
                                  log_dir=os.path.join(home,'runs'),
                                  save_path=os.path.join(home, 'modelds'))

[INFO] Created SummaryWriter, saving to: /home/ayhem18/DEV/My_Kaggle_Repo/fashionMnist/runs/experience_6...


  4%|▍         | 1/25 [00:02<01:04,  2.68s/it]

#########################
training loss: 0.1939176157826469
train_accuracy: 0.9275855654761904
validation loss : 0.8929219345251719
val_accuracy: 0.7359813749790192
#########################


 24%|██▍       | 6/25 [00:15<00:50,  2.65s/it]

#########################
training loss: 0.14240382178908303
train_accuracy: 0.9475074404761905
validation loss : 0.34357859939336777
val_accuracy: 0.8848788489898046
#########################


...

#########################
training loss: 0.09337055807312329
train_accuracy: 0.9647693452380952
validation loss : 0.3947496364514033
val_accuracy: 0.8894998331864675
#########################


 84%|████████▍ | 21/25 [00:55<00:10,  2.65s/it]

#########################
training loss: 0.06849331811425231
train_accuracy: 0.9747767857142857
validation loss : 0.34883206834395725
val_accuracy: 0.9043747186660767
#########################


100%|██████████| 25/25 [01:06<00:00,  2.66s/it]


{'train_loss': [0.1939176157826469,
  0.17453499706018538,
  0.16918457554919378,
  0.15803656372286023,
  0.1553766144883065,
  0.14240382178908303,
  0.14268162718841007,
  0.1348015762510754,
  0.12736526415461585,
  0.12173406510126022,
  0.1172335874466669,
  0.1146486120564597,
  0.10249199973685401,
  0.09666735671815418,
  0.09239702267306192,
  0.09337055807312329,
  0.08404433209271658,
  0.0759745377160254,
  0.07369048027765183,
  0.07470139995926903,
  0.06849331811425231,
  0.06583823751480807,
  0.0599398761278107,
  0.05903842447414285,
  0.05127636809788999],
 'val_loss': [0.8929219345251719,
  0.3610270942250888,
  0.4890676811337471,
  0.30484892552097637,
  0.6841419786214828,
  0.34357859939336777,
  0.43326984345912933,
  0.29298224796851474,
  0.5401079282164574,
  0.3155454881489277,
  0.35516347736120224,
  0.6624443878730139,
  0.3716977499425411,
  0.31419986858963966,
  0.3339410225550334,
  0.3947496364514033,
  0.3763350546360016,
  0.32325854897499084,
  

Split the dataset into training and validation subsets using `random_split`

# Inference

In [30]:
# generate the submission file
predictions = engine_classification.inference(model, test_loader, return_tensor='list')
sub_folder = os.path.join(home, 'submissions')
submission_df = pd.DataFrame(columns=['id', 'label'])
submission_df['id'] = list(range(60000, 60000 + len(predictions)))
submission_df['label'] = predictions
submission_df.to_csv(os.path.join(home, sub_folder, f'{len(os.listdir(sub_folder))}.csv'), index=None)